# 000 preprocess
---

In [2]:
import os, gc, yaml, glob, pickle
import time
import random
from pathlib import Path
from types import SimpleNamespace

import numpy as np
import polars as pl
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append("../src/utils")
from metric import score
from data import sep, show_df, glob_walk, set_seed, save_config_yaml, dict_to_namespace

In [4]:
# ===============
# Config
# ===============
config = SimpleNamespace(
    INPUT_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw"),
    TRAIN_TRACKING_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "train_tracking",
    TRAIN_ANNOTATION_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "train_annotation",
    TEST_TRACKING_DIR = Path("/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw") / "test_tracking",
    seed = 42,
    n_splits = 5,
    INDEX_COLS = [
        "video_id",
        "agent_mouse_id",
        "target_mouse_id",
        "video_frame",
    ],
    BODY_PARTS = [
        "ear_left",
        "ear_right",
        "nose",
        "neck",
        "body_center",
        "lateral_left",
        "lateral_right",
        "hip_left",
        "hip_right",
        "tail_base",
        "tail_tip",
    ],
    SELF_BEHAVIORS = [
        "biteobject",
        "climb",
        "dig",
        "exploreobject",
        "freeze",
        "genitalgroom",
        "huddle",
        "rear",
        "rest",
        "run",
        "selfgroom",
    ],
    PAIR_BEHAVIORS = [
        "allogroom",
        "approach",
        "attack",
        "attemptmount",
        "avoid",
        "chase",
        "chaseattack",
        "defend",
        "disengage",
        "dominance",
        "dominancegroom",
        "dominancemount",
        "ejaculate",
        "escape",
        "flinch",
        "follow",
        "intromit",
        "mount",
        "reciprocalsniff",
        "shepherd",
        "sniff",
        "sniffbody",
        "sniffface",
        "sniffgenital",
        "submit",
        "tussle",
    ],
)

In [5]:
# ===============
# read data
# ===============
train_dataframe = pl.read_csv(config.INPUT_DIR / "train.csv")
sep("train_dataframe"); show_df(train_dataframe)

train_dataframe
(8789, 38)


lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
str,i64,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,f64,f64,f64,i64,i64,f64,f64,str,str,str,str,str
"""AdaptableSnail""",44566106,"""CD-1 (ICR)""","""white""","""male""",10.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",24.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",38.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",51.0,"""8-12 weeks""","""wireless device""",30.0,615.6,16.0,1228,1068,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""
"""AdaptableSnail""",143861384,"""CD-1 (ICR)""","""white""","""male""",3.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",17.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",31.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",44.0,"""8-12 weeks""",null,25.0,3599.0,9.7,968,608,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""
"""AdaptableSnail""",209576908,"""CD-1 (ICR)""","""white""","""male""",7.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",21.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",35.0,"""8-12 weeks""",null,"""CD-1 (ICR)""","""white""","""male""",48.0,"""8-12 weeks""",null,30.0,615.2,16.0,1266,1100,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""


In [23]:
# test
test_dataframe = pl.read_csv(config.INPUT_DIR / "test.csv")
sep("test_dataframe"); show_df(test_dataframe)

# sub
sub_dataframe = pl.read_csv(config.INPUT_DIR / "sample_submission.csv")
sep("sub_dataframe"); show_df(sub_dataframe)

test_dataframe
(1, 38)


lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
str,i64,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,str,str,str,f64,str,str,f64,f64,f64,i64,i64,f64,f64,str,str,str,str,str
"""AdaptableSnail""",438887472,"""CD-1 (ICR)""","""white""","""male""",13.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",27.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",41.0,"""8-12 weeks""","""wireless device""","""CD-1 (ICR)""","""white""","""male""",53.0,"""8-12 weeks""","""wireless device""",30.0,614.7,16.0,1214,1090,60.0,60.0,"""square""","""familiar""","""[""body_center"", ""ear_left"", ""e…","""[""mouse1,mouse2,approach"", ""mo…","""DeepLabCut"""


sub_dataframe
(1, 7)


row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
i64,i64,str,str,str,i64,i64
0,438887472,"""mouse1""","""mouse2""","""sniff""",0,1


In [ ]:
train_dataframe["video_id"].unique().len() # 全てユニークな動画ID

8789

In [20]:
train_dataframe.filter(pl.col("behaviors_labeled").is_null()).shape
# 7941/8789行がラベルなし

(7941, 38)

In [21]:
train_dataframe.filter(pl.col("behaviors_labeled").is_not_null()).shape

(848, 38)

In [ ]:
#　プロセス確認用
# (train_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
# .select(
#     pl.col("lab_id"),
#     pl.col("video_id"),
#     pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
# )
#     .explode("behaviors_labeled_list")
# )

lab_id,video_id,behaviors_labeled_list
str,i64,str
"""AdaptableSnail""",44566106,"""mouse1,mouse2,approach"""
"""AdaptableSnail""",44566106,"""mouse1,mouse2,attack"""
"""AdaptableSnail""",44566106,"""mouse1,mouse2,avoid"""
"""AdaptableSnail""",44566106,"""mouse1,mouse2,chase"""
"""AdaptableSnail""",44566106,"""mouse1,mouse2,chaseattack"""
…,…,…
"""UppityFerret""",2104375029,"""mouse1,mouse2,sniff"""
"""UppityFerret""",2104375029,"""mouse1,mouse2,sniffgenital"""
"""UppityFerret""",2104375029,"""mouse2,mouse1,sniffgenital"""


In [ ]:
# =============================
# preprocess behavior labels
# =============================
train_behavior_dataframe = (
    train_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        # list化(["mouse1,mouse2,approach", "mouse1,mouse2,attack", … "mouse4,self,rear"])
        pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
    )
    # listをexplodeして1行ずつに分解(848->4_919行)
    .explode("behaviors_labeled_list")
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled_element").str.split(",").list[0].str.replace_all("'", "").alias("agent"),
        pl.col("behaviors_labeled_element").str.split(",").list[1].str.replace_all("'", "").alias("target"),
        pl.col("behaviors_labeled_element").str.split(",").list[2].str.replace_all("'", "").alias("behavior"),
    )
)

train_self_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(config.SELF_BEHAVIORS))
train_pair_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(config.PAIR_BEHAVIORS))
sep("train_self_behavior_dataframe")
show_df(train_self_behavior_dataframe)
sep("train_pair_behavior_dataframe")
show_df(train_pair_behavior_dataframe)

train_self_behavior_dataframe
(594, 5)


lab_id,video_id,agent,target,behavior
str,i64,str,str,str
"""AdaptableSnail""",44566106,"""mouse1""","""self""","""rear"""
"""AdaptableSnail""",44566106,"""mouse2""","""self""","""rear"""
"""AdaptableSnail""",44566106,"""mouse3""","""self""","""rear"""


train_pair_behavior_dataframe
(4325, 5)


lab_id,video_id,agent,target,behavior
str,i64,str,str,str
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""approach"""
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""attack"""
"""AdaptableSnail""",44566106,"""mouse1""","""mouse2""","""avoid"""


# Tracking data

In [32]:
# =============================
# gather tracking paths
# =============================
tracking_paths = glob_walk(config.TRAIN_TRACKING_DIR, "**/*.parquet")
print(f"length of tracking paths: {len(tracking_paths)}")
print(tracking_paths[:3])

length of tracking paths: 8789
[PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1212811043.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1260392287.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_tracking/AdaptableSnail/1351098077.parquet')]


In [46]:
df_tracking_sample = pl.read_parquet(tracking_paths[2])
sep("df_tracking_sample"); show_df(df_tracking_sample)

df_tracking_sample
(1617776, 5)


video_frame,mouse_id,bodypart,x,y
i32,i8,str,f32,f32
0,1,"""body_center""",1149.209961,445.333008
0,1,"""ear_left""",1157.660034,516.973999
0,1,"""ear_right""",1116.16394,485.903015


In [47]:
print(*list(df_tracking_sample["video_frame"].unique())[:10])

0 1 2 3 4 5 6 7 8 9


In [55]:
df_pivot = (
    df_tracking_sample
    .pivot(
        index=["video_frame", "mouse_id"],
        on="bodypart",      
        values=["x", "y"],
    )
    .sort(["video_frame", "mouse_id"])
)
sep("df_pivot"); show_df(df_pivot, 10, True)

# 2_677_006 -> 318_646行に削減

df_pivot
(242707, 22)


video_frame,mouse_id,x_body_center,x_ear_left,x_ear_right,x_lateral_left,x_lateral_right,x_nose,x_tail_base,x_tail_tip,x_tail_midpoint,x_neck,y_body_center,y_ear_left,y_ear_right,y_lateral_left,y_lateral_right,y_nose,y_tail_base,y_tail_tip,y_tail_midpoint,y_neck
i32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1,1149.209961,1157.660034,1116.16394,1172.079956,1132.894043,1116.237061,1165.696045,330.029999,null,null,445.333008,516.973999,485.903015,476.265015,398.407013,526.552002,389.093994,367.631989,null,null
0,2,1104.375977,1123.755981,1072.952026,1128.312988,1075.534058,1110.120972,1081.697998,980.187012,1030.911011,null,304.947998,332.627014,342.309998,290.011993,308.338989,380.158997,274.985992,188.085999,234.401993,null
0,4,264.707001,245.007996,237.647995,284.531006,261.359009,188.632996,295.811005,null,342.378998,null,529.129028,561.588013,517.10199,554.869019,495.196014,538.15802,503.34201,null,447.311005,null
1,1,1151.474976,1152.540039,1138.628052,1171.180054,1164.267944,1102.490967,1163.123047,330.640991,null,null,452.80899,521.947021,405.372986,482.645996,402.563995,383.053986,396.161011,367.600006,null,null
1,2,1105.51001,1124.618042,1072.762939,1134.557007,1082.232056,null,1094.462036,980.932007,1047.628052,null,315.009003,336.235992,340.915985,312.863007,302.312988,null,280.283997,194.697006,231.712006,null
1,4,269.755005,246.979004,241.117996,291.902008,264.782013,196.436996,303.700989,null,346.834991,null,527.312988,562.159973,521.320007,546.619995,492.790009,527.757019,502.001007,null,446.829987,null
2,1,1151.827026,1153.832031,1132.987061,1171.368042,1156.902954,null,1165.732056,330.735992,null,null,453.035004,520.281982,405.522003,482.233002,403.054993,null,397.067993,369.046997,null,null
2,2,1103.655029,1119.79895,1068.567017,1132.973022,1075.192993,1099.938965,1094.609985,981.106995,1048.917969,null,314.234985,335.475006,340.154999,316.86499,305.850006,385.057007,281.042999,194.479004,231.863007,null
2,4,273.295013,248.285995,246.292007,287.394989,271.584991,196.388,304.661987,null,348.813995,null,527.046997,558.210022,520.366028,548.804993,495.036987,517.812012,502.984985,null,439.544006,null


video_frame,mouse_id,x_body_center,x_ear_left,x_ear_right,x_lateral_left,x_lateral_right,x_nose,x_tail_base,x_tail_tip,x_tail_midpoint,x_neck,y_body_center,y_ear_left,y_ear_right,y_lateral_left,y_lateral_right,y_nose,y_tail_base,y_tail_tip,y_tail_midpoint,y_neck
i32,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
81776,4,635.982971,620.5,617.804016,null,null,646.705017,null,608.919983,null,null,101.098,110.539001,111.748001,null,null,156.102997,null,163.878998,null,null
81777,1,602.534973,null,null,620.192017,597.559998,null,574.695984,564.442993,557.835999,null,68.638,null,null,82.982002,89.660004,null,77.628998,108.317001,126.516998,null
81777,2,581.731995,617.210999,575.77301,614.906982,552.515991,622.234985,579.573975,null,594.070984,null,927.046997,938.16803,972.663025,910.075989,937.380981,999.974976,890.33197,null,817.927979,null
81777,4,635.952026,620.580017,617.700012,null,null,646.632019,null,609.286987,null,null,100.966003,110.718002,111.996002,null,null,155.968002,null,163.867004,null,null
81778,1,602.383972,null,null,620.070984,597.575989,null,574.690979,564.47699,557.830994,null,68.906998,null,null,83.053001,89.403,null,77.626999,108.266998,126.473,null
81778,2,581.960999,619.184021,575.815979,615.651978,550.783997,621.140991,579.622986,null,594.093994,null,927.250977,940.935974,972.817993,909.927979,930.218994,999.776978,890.445007,null,817.734985,null
81778,4,635.971985,620.562988,617.469971,null,null,646.69397,null,609.655029,null,null,101.106003,110.769997,112.200996,null,null,155.854996,null,163.785995,null,null
81779,1,602.387024,null,null,620.077026,597.637024,605.573975,574.737976,564.504028,557.848022,null,68.956001,null,null,83.086998,89.438004,106.528,77.634003,108.267998,126.732002,null
81779,2,581.28302,619.11499,575.786011,616.012024,551.484985,620.854004,579.580017,null,594.012024,null,926.465027,940.682983,972.708008,910.296997,931.075012,999.987976,890.598999,null,818.166016,null


In [58]:
def rename_agent(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_agent_" + col[2:]   # "x_" の2文字を切って後ろだけ使う
    if col.startswith("y_"):
        return "y_agent_" + col[2:]
    return col

def rename_target(col: str) -> str:
    if col in ("video_frame", "mouse_id"):
        return col
    if col.startswith("x_"):
        return "x_target_" + col[2:]
    if col.startswith("y_"):
        return "y_target_" + col[2:]
    return col

In [61]:
def make_pair_features(df_pivot: pl.DataFrame, agent_id: int, target_id: int) -> pl.DataFrame:
    # agentの系列
    agent = (
        df_pivot
        .filter(pl.col("mouse_id") == agent_id)
        .rename(rename_agent)
    )
    # targetの系列
    target = (
        df_pivot
        .filter(pl.col("mouse_id") == target_id)
        .rename(rename_target)
    )
    # video_frame で結合
    pair = (
        agent.drop("mouse_id")
        .join(target.drop("mouse_id"), on="video_frame", how="inner")
        .sort("video_frame")
    )
    # show_df(pair, 10, True)
    # 相対位置・距離などを追加
    pair = pair.with_columns(
        (pl.col("x_target_body_center") - pl.col("x_agent_body_center")).alias("dx"),
        (pl.col("y_target_body_center") - pl.col("y_agent_body_center")).alias("dy"),
    ).with_columns(
        (pl.col("dx") ** 2 + pl.col("dy") ** 2).sqrt().alias("dist"),
    )
    return pair

In [62]:
df_tracking_sample_pair = make_pair_features(df_pivot, agent_id=1, target_id=2)
sep("df_tracking_sample_pair"); show_df(df_tracking_sample_pair, 10, True)

df_tracking_sample_pair
(80062, 44)


video_frame,x_agent_body_center,x_agent_ear_left,x_agent_ear_right,x_agent_lateral_left,x_agent_lateral_right,x_agent_nose,x_agent_tail_base,x_agent_tail_tip,x_agent_tail_midpoint,x_agent_neck,y_agent_body_center,y_agent_ear_left,y_agent_ear_right,y_agent_lateral_left,y_agent_lateral_right,y_agent_nose,y_agent_tail_base,y_agent_tail_tip,y_agent_tail_midpoint,y_agent_neck,x_target_body_center,x_target_ear_left,x_target_ear_right,x_target_lateral_left,x_target_lateral_right,x_target_nose,x_target_tail_base,x_target_tail_tip,x_target_tail_midpoint,x_target_neck,y_target_body_center,y_target_ear_left,y_target_ear_right,y_target_lateral_left,y_target_lateral_right,y_target_nose,y_target_tail_base,y_target_tail_tip,y_target_tail_midpoint,y_target_neck,dx,dy,dist
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,1149.209961,1157.660034,1116.16394,1172.079956,1132.894043,1116.237061,1165.696045,330.029999,null,null,445.333008,516.973999,485.903015,476.265015,398.407013,526.552002,389.093994,367.631989,null,null,1104.375977,1123.755981,1072.952026,1128.312988,1075.534058,1110.120972,1081.697998,980.187012,1030.911011,null,304.947998,332.627014,342.309998,290.011993,308.338989,380.158997,274.985992,188.085999,234.401993,null,-44.833984,-140.38501,147.370407
1,1151.474976,1152.540039,1138.628052,1171.180054,1164.267944,1102.490967,1163.123047,330.640991,null,null,452.80899,521.947021,405.372986,482.645996,402.563995,383.053986,396.161011,367.600006,null,null,1105.51001,1124.618042,1072.762939,1134.557007,1082.232056,null,1094.462036,980.932007,1047.628052,null,315.009003,336.235992,340.915985,312.863007,302.312988,null,280.283997,194.697006,231.712006,null,-45.964966,-137.799988,145.263947
2,1151.827026,1153.832031,1132.987061,1171.368042,1156.902954,null,1165.732056,330.735992,null,null,453.035004,520.281982,405.522003,482.233002,403.054993,null,397.067993,369.046997,null,null,1103.655029,1119.79895,1068.567017,1132.973022,1075.192993,1099.938965,1094.609985,981.106995,1048.917969,null,314.234985,335.475006,340.154999,316.86499,305.850006,385.057007,281.042999,194.479004,231.863007,null,-48.171997,-138.800018,146.921707
3,1151.515991,1151.015015,1133.811035,1170.697021,1156.859009,null,1165.370972,330.743988,null,null,453.364014,520.856995,405.924011,483.083008,402.816986,null,396.713013,367.914001,null,null,1098.209961,1119.234009,1064.43103,1131.587036,1070.720947,1097.875,1093.571045,979.241028,1046.725952,null,318.411987,340.132996,346.139008,314.371002,311.549988,391.467987,280.559998,195.647003,230.904007,null,-53.30603,-134.952026,145.098526
4,1150.463989,1150.425049,1136.776001,1170.224976,1132.267944,null,1165.357056,null,null,null,454.019012,521.846985,403.516998,483.425995,453.058014,null,396.131989,null,null,null,1095.712036,1115.084961,1058.192017,1125.430054,1070.906006,1092.083008,1091.31897,973.208984,1047.359009,null,320.815002,341.51001,350.02301,306.688995,313.287994,397.033997,284.833008,188.957993,231.783997,null,-54.751953,-133.20401,144.017654
5,1150.41394,1148.480957,1135.156006,1169.813965,null,null,1164.758057,null,null,null,453.595001,523.04303,405.962006,484.686005,null,null,396.13501,null,null,null,1094.618042,1114.537964,1054.892944,1124.831055,1070.761963,1093.89502,1089.692993,980.336975,1048.54895,null,320.144989,343.378998,354.856995,306.085999,315.998993,394.795013,286.595001,193.626007,232.919006,null,-55.795898,-133.450012,144.644699
6,1150.467041,1148.758057,1136.973022,1170.001953,1124.781982,null,1165.926025,null,1158.162964,null,455.716003,523.242981,406.184998,484.928009,459.809998,null,397.696991,null,325.351013,null,1095.182983,1112.090942,1056.958008,1125.233032,1070.355957,1096.082031,1090.44397,978.117004,1048.109009,null,320.196014,342.21701,357.438995,306.28299,314.829987,395.584015,286.847992,189.291,232.798996,null,-55.284058,-135.519989,146.362549
7,1149.87902

video_frame,x_agent_body_center,x_agent_ear_left,x_agent_ear_right,x_agent_lateral_left,x_agent_lateral_right,x_agent_nose,x_agent_tail_base,x_agent_tail_tip,x_agent_tail_midpoint,x_agent_neck,y_agent_body_center,y_agent_ear_left,y_agent_ear_right,y_agent_lateral_left,y_agent_lateral_right,y_agent_nose,y_agent_tail_base,y_agent_tail_tip,y_agent_tail_midpoint,y_agent_neck,x_target_body_center,x_target_ear_left,x_target_ear_right,x_target_lateral_left,x_target_lateral_right,x_target_nose,x_target_tail_base,x_target_tail_tip,x_target_tail_midpoint,x_target_neck,y_target_body_center,y_target_ear_left,y_target_ear_right,y_target_lateral_left,y_target_lateral_right,y_target_nose,y_target_tail_base,y_target_tail_tip,y_target_tail_midpoint,y_target_neck,dx,dy,dist
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
81770,602.614014,null,null,596.330994,597.541016,605.629028,575.034973,564.231995,558.41803,null,68.438004,null,null,51.455002,89.972,106.583,77.025002,108.372002,125.962997,null,581.877014,614.080994,572.971008,614.033997,552.494995,620.247986,579.781006,null,593.757019,null,925.432007,937.112976,969.551025,909.768005,936.557983,992.491028,890.362,null,818.609985,null,-20.737,856.994019,857.244873
81771,602.771973,null,null,620.156982,597.541016,605.585022,574.906982,564.294983,558.375,null,68.296997,null,null,83.035004,89.670998,106.564003,77.223999,108.408997,126.110001,null,581.892029,615.862976,574.892029,612.877014,552.374023,620.429016,579.52301,null,593.976013,null,925.815979,937.91803,970.252014,909.60199,938.219971,992.669983,890.067017,null,817.979004,null,-20.879944,857.518982,857.773193
81772,602.896973,null,null,620.151978,597.650024,605.66803,574.893982,564.338989,558.267029,null,68.325996,null,null,83.078003,89.689003,106.547997,77.399002,108.431999,126.806,null,581.903015,616.036987,573.86499,612.97998,550.317017,619.976013,579.525024,null,593.91803,null,925.749023,938.067993,968.896973,909.343018,931.127991,993.698975,890.05603,null,818.229004,null,-20.993958,857.423035,857.679993
81773,602.89502,null,null,596.434021,597.476013,605.596008,574.801025,564.333984,558.210999,null,68.286003,null,null,51.636002,89.633003,106.607002,77.275002,108.416,126.753998,null,581.067017,615.328003,574.169983,612.857971,549.98999,620.377991,579.619019,null,593.982971,null,925.481018,937.336975,970.64801,910.213013,930.841003,993.505981,890.096985,null,817.945007,null,-21.828003,857.195007,857.472839
81774,602.71698,null,null,596.494995,597.362976,605.572998,574.674988,564.38501,558.218018,null,68.481003,null,null,51.682999,89.497002,106.589996,77.166,108.402,126.305,null,581.455017,617.435974,574.135986,613.39502,550.577026,620.429016,579.986023,null,594.02301,null,926.21698,936.580017,970.247986,910.52002,931.744995,994.489014,890.153015,null,817.642029,null,-21.261963,857.735962,857.999451
81775,602.741028,null,null,620.182007,595.492004,605.651001,574.768982,564.398987,558.060974,null,68.639999,null,null,82.999001,94.714996,106.435997,77.571999,108.408997,126.475998,null,581.336975,616.856018,575.364014,613.880981,550.901978,622.231018,579.583984,null,594.156982,null,926.731018,937.192993,972.070007,909.742981,929.947021,997.468994,890.312012,null,817.838989,null,-21.404053,858.091003,858.35791
81776,602.661011,null,null,620.239014,597.448975,null,574.502014,564.424011,557.757996,null,68.542999,null,null,82.915001,89.639999,null,77.473999,108.346001,126.721001,null,580.950012,617.349976,575.271973,614.304016,551.948975,622.036011,579.630981,null,594.158997,null,926.630981,937.377991,971.585022,910.08197,936.455017,997.242004,890.247009,null,817.869019,null,-21.710999,858.088013,858.362671
81777,602.534973,null,null,620.192017,597.559998,null,574.695984,564.442993,557.835999,null,68.638,null,null,82.982002,89.660004,null,77.628998,108.317001,126.516998,null,581.731995,617.210999,575.77301,614.906982,

# Annotation data

In [39]:
# =============================
# gather annotation paths
# =============================
annotation_paths = glob_walk(config.TRAIN_ANNOTATION_DIR, "**/*.parquet")
print(f"length of annotation paths: {len(annotation_paths)}")
print(annotation_paths[:3])

length of annotation paths: 847
[PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1212811043.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1260392287.parquet'), PosixPath('/mnt/nfs/home/hidebu/study/MABe-Challenge---Social-Action-Recognition-in-Mice/data/raw/train_annotation/AdaptableSnail/1351098077.parquet')]


In [40]:
df_annotation_sample = pl.read_parquet(annotation_paths[0])
sep("df_annotation_sample"); show_df(df_annotation_sample)

df_annotation_sample
(370, 5)


agent_id,target_id,action,start_frame,stop_frame
i8,i8,str,i32,i32
1,3,"""chase""",2,54
1,3,"""chase""",128,234
3,2,"""avoid""",324,342
